In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()


In [ ]:
GROQ_API_KEY=os.getenv('GROQ_API_KEY')


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('attention.pdf')
docs=loader.load()


In [ ]:
docs


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)


In [ ]:
len(documents)


In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_groq import ChatGroq


In [ ]:
llm=ChatGroq(groq_api_key=GROQ_API_KEY,
         model_name="qwen/qwen3-32b")



In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>

Question: {input}""")


In [ ]:
db=Chroma.from_documents(documents[:10],OllamaEmbeddings(model="llama2"))


In [ ]:
db


In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)


In [ ]:
retriever=db.as_retriever()
retriever
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [ ]:
response=retrieval_chain.invoke({"input":"what is attention?"})


In [ ]:
print(response)


